In [0]:
# Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
downloaded = drive.CreateFile({'id':'1V1Z2C2mhCejIdF55dzShs2GvkF3JiEjh'}) # replace the id with id of file you want to access
downloaded.GetContentFile('npr.csv') 

In [4]:
import pandas as pd
data = pd.read_table('npr.csv', sep=",") 
print(data.head())
print(data.describe)

                                             Article
0  In the Washington of 2016, even when the polic...
1    Donald Trump has used Twitter  —   his prefe...
2    Donald Trump is unabashedly praising Russian...
3  Updated at 2:50 p. m. ET, Russian President Vl...
4  From photography, illustration and video, to d...
<bound method NDFrame.describe of                                                  Article
0      In the Washington of 2016, even when the polic...
1        Donald Trump has used Twitter  —   his prefe...
2        Donald Trump is unabashedly praising Russian...
3      Updated at 2:50 p. m. ET, Russian President Vl...
4      From photography, illustration and video, to d...
...                                                  ...
11987  The number of law enforcement officers shot an...
11988    Trump is busy these days with victory tours,...
11989  It’s always interesting for the Goats and Soda...
11990  The election of Donald Trump was a surprise to...
11991  Voters in the 

In [9]:
data.isnull().sum()
#data.dropna(inplace = True)
indexes=[]
for i,article in data.itertuples():
  if article.isspace():
    indexes.append[i]
data.drop(indexes, inplace = True)
print(data.shape)

(11992, 1)


In [13]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_df = 0.9, min_df = 2, stop_words = 'english')
#trans_x = cv.fit_transform(x.values.astype('U'))# to convert from object to unicode
trans_x = cv.fit_transform(data["Article"])
trans_x.shape

(11992, 54777)

In [0]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components = 7, random_state=42)
output = lda.fit(trans_x)

In [15]:
output.components_.shape

(7, 54777)

In [16]:
single_topic = output.components_[2]
print(single_topic.shape)
top_twenty_frequent_word_indexes = single_topic.argsort()[-20:]
print(top_twenty_frequent_word_indexes)

(54777,)
[28875 27439 15008 54403 29746 53211 54102 18079 23401 12823 49459 53152
  9511 33390 54412 19307 26752 36283 28659 42993]


In [17]:
for index in top_twenty_frequent_word_indexes:
  print(cv.get_feature_names()[index])

little
know
don
year
make
way
world
family
home
day
time
water
city
new
years
food
just
people
like
says


In [22]:
for topic, words in enumerate(output.components_):
  print(f"here are top 15 frquency words from topic{topic}")
  print([cv.get_feature_names()[index] for index in words.argsort()[-20:]])
  print("\n")
  print("\n")
  #0 = money,1 = governement,2=general,3=health,4=elections,5=entertainment,6= education,

here are top 15 frquency words from topic0
['president', 'state', 'tax', 'insurance', 'trump', 'companies', 'money', 'year', 'federal', '000', 'new', 'percent', 'government', 'company', 'million', 'care', 'people', 'health', 'said', 'says']




here are top 15 frquency words from topic1
['white', 'according', 'attack', 'reported', 'war', 'military', 'house', 'security', 'russia', 'government', 'npr', 'reports', 'says', 'news', 'people', 'told', 'police', 'president', 'trump', 'said']




here are top 15 frquency words from topic2
['little', 'know', 'don', 'year', 'make', 'way', 'world', 'family', 'home', 'day', 'time', 'water', 'city', 'new', 'years', 'food', 'just', 'people', 'like', 'says']




here are top 15 frquency words from topic3
['world', 'research', 'university', 'percent', 'care', 'time', 'new', 'don', 'years', 'medical', 'disease', 'patients', 'just', 'children', 'study', 'like', 'women', 'health', 'people', 'says']




here are top 15 frquency words from topic4
['donald',

In [19]:
results = output.transform(trans_x)
print(results.shape)

(11992, 7)


In [21]:
data["topics"] = results.argmax(axis =1)
print(data)

                                                 Article  topics
0      In the Washington of 2016, even when the polic...       1
1        Donald Trump has used Twitter  —   his prefe...       1
2        Donald Trump is unabashedly praising Russian...       1
3      Updated at 2:50 p. m. ET, Russian President Vl...       1
4      From photography, illustration and video, to d...       2
...                                                  ...     ...
11987  The number of law enforcement officers shot an...       1
11988    Trump is busy these days with victory tours,...       4
11989  It’s always interesting for the Goats and Soda...       3
11990  The election of Donald Trump was a surprise to...       4
11991  Voters in the English city of Sunderland did s...       0

[11992 rows x 2 columns]


In [27]:
final_topic = results.argmax(axis =1)
replace_values = {0 : "Money", 1 : "Governement", 2 : "General", 3 : "Health", 4 : "Election", 5 : "Entertainment", 6 : "Education" } 
#0 = money,1 = governement,2=general,3=health,4=elections,5=entertainment,6= education,
data = data.replace({"topics": replace_values})  
data

,Article,topics
0,"In the Washington of 2016, even when the polic...",Governement
1,Donald Trump has used Twitter — his prefe...,Governement
2,Donald Trump is unabashedly praising Russian...,Governement
3,"Updated at 2:50 p. m. ET, Russian President Vl...",Governement
4,"From photography, illustration and video, to d...",General
...,...,...
11987,The number of law enforcement officers shot an...,Governement
11988,"Trump is busy these days with victory tours,...",Election
11989,It’s always interesting for the Goats and Soda...,Health
11990,The election of Donald Trump was a surprise to...,Election
